<a href="https://colab.research.google.com/github/mimilazarova/dd2412_project_fixmatch_and_beyond/blob/main/WRN_28_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
from tqdm import tqdm, tqdm_notebook

def regularized_padded_conv(*args, **kwargs):
    return tf.keras.layers.Conv2D(*args, **kwargs, padding='same', kernel_regularizer=_regularizer,
                                  kernel_initializer='he_normal', use_bias=False)


def BN_ReLU(x):
    x = tf.keras.layers.BatchNormalization()(x)
    return tf.keras.layers.ReLU()(x)


def shortcut(x, filters, stride, mode):
    if x.shape[-1] == filters:
        return x
    elif mode == 'B':
        return regularized_padded_conv(filters, 1, strides=stride)(x)
    elif mode == 'B_original':
        x = regularized_padded_conv(filters, 1, strides=stride)(x)
        return tf.keras.layers.BatchNormalization()(x)
    elif mode == 'A':
        return tf.pad(tf.keras.layers.MaxPool2D(1, stride)(x) if stride>1 else x,
                      paddings=[(0, 0), (0, 0), (0, 0), (0, filters - x.shape[-1])])
    else:
        raise KeyError("Parameter shortcut_type not recognized!")
    

def original_block(x, filters, stride=1, **kwargs):
    c1 = regularized_padded_conv(filters, 3, strides=stride)(x)
    c2 = regularized_padded_conv(filters, 3)(BN_ReLU(c1))
    c2 = tf.keras.layers.BatchNormalization()(c2)
    
    mode = 'B_original' if _shortcut_type == 'B' else _shortcut_type
    x = shortcut(x, filters, stride, mode=mode)
    return tf.keras.layers.ReLU()(x + c2)
    
    
def preactivation_block(x, filters, stride=1, preact_block=False):
    flow = BN_ReLU(x)
    if preact_block:
        x = flow
        
    c1 = regularized_padded_conv(filters, 3, strides=stride)(flow)
    if _dropout:
        c1 = tf.keras.layers.Dropout(_dropout)(c1)
        
    c2 = regularized_padded_conv(filters, 3)(BN_ReLU(c1))
    x = shortcut(x, filters, stride, mode=_shortcut_type)
    return x + c2


def bootleneck_block(x, filters, stride=1, preact_block=False):
    flow = BN_ReLU(x)
    if preact_block:
        x = flow
         
    c1 = regularized_padded_conv(filters//_bootleneck_width, 1)(flow)
    c2 = regularized_padded_conv(filters//_bootleneck_width, 3, strides=stride)(BN_ReLU(c1))
    c3 = regularized_padded_conv(filters, 1)(BN_ReLU(c2))
    x = shortcut(x, filters, stride, mode=_shortcut_type)
    return x + c3


def group_of_blocks(x, block_type, num_blocks, filters, stride, block_idx=0):
    global _preact_shortcuts
    preact_block = True if _preact_shortcuts or block_idx == 0 else False
    
    x = block_type(x, filters, stride, preact_block=preact_block)
    for i in range(num_blocks-1):
        x = block_type(x, filters)
    return x


def Resnet(input_shape, n_classes, l2_reg=1e-4, group_sizes=(2, 2, 2), features=(16, 32, 64), strides=(1, 2, 2),
           shortcut_type='B', block_type='preactivated', first_conv={"filters": 16, "kernel_size": 3, "strides": 1},
           dropout=0, cardinality=1, bootleneck_width=4, preact_shortcuts=True):
    
    global _regularizer, _shortcut_type, _preact_projection, _dropout, _cardinality, _bootleneck_width, _preact_shortcuts
    _bootleneck_width = bootleneck_width # used in ResNeXts and bootleneck blocks
    _regularizer = tf.keras.regularizers.l2(l2_reg)
    _shortcut_type = shortcut_type # used in blocks
    _cardinality = cardinality # used in ResNeXts
    _dropout = dropout # used in Wide ResNets
    _preact_shortcuts = preact_shortcuts
    
    block_types = {'preactivated': preactivation_block,
                   'bootleneck': bootleneck_block,
                   'original': original_block}
    
    selected_block = block_types[block_type]
    inputs = tf.keras.layers.Input(shape=input_shape)
    flow = regularized_padded_conv(**first_conv)(inputs)
    
    if block_type == 'original':
        flow = BN_ReLU(flow)
    
    for block_idx, (group_size, feature, stride) in enumerate(zip(group_sizes, features, strides)):
        flow = group_of_blocks(flow,
                               block_type=selected_block,
                               num_blocks=group_size,
                               block_idx=block_idx,
                               filters=feature,
                               stride=stride)
    
    if block_type != 'original':
        flow = BN_ReLU(flow)
    
    flow = tf.keras.layers.GlobalAveragePooling2D()(flow)
    outputs = tf.keras.layers.Dense(n_classes, kernel_regularizer=_regularizer)(flow)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


# If we want to load pre-trained weights
def load_weights_func(model, model_name):
    try: model.load_weights(os.path.join('saved_models', model_name + '.tf'))
    except tf.errors.NotFoundError: print("No weights found for this model!")
    return model

# Creating a wide ResNet for cifar-10 dataset
def cifar_wide_resnet(N, K, block_type='preactivated', shortcut_type='B', dropout=0, l2_reg=2.5e-4):
    assert (N-4) % 6 == 0, "N-4 has to be divisible by 6"
    lpb = (N-4) // 6 # layers per block - since N is total number of convolutional layers in Wide ResNet
    model = Resnet(input_shape=(32, 32, 3), n_classes=10, l2_reg=l2_reg, group_sizes=(lpb, lpb, lpb), features=(16*K, 32*K, 64*K),
                   strides=(1, 2, 2), first_conv={"filters": 16, "kernel_size": 3, "strides": 1}, shortcut_type=shortcut_type,
                   block_type=block_type, dropout=dropout, preact_shortcuts=True)
    return model

# Specifying wide ResNet-28-2 model
def WRN_28_2(shortcut_type='B', load_weights=False, dropout=0, l2_reg=2.5e-4):
    model = cifar_wide_resnet(28, 2, 'preactivated', shortcut_type, dropout=dropout, l2_reg=l2_reg)
    if load_weights: model = load_weights_func(model, 'cifar_WRN_28_10') # loading pre-trained weights
    return model


In [2]:
model = WRN_28_2()

In [3]:
# Load CIFAR-10 dataset
ds = tfds.load('cifar10', as_supervised=True, in_memory=True)
#std = tf.reshape((0.2023, 0.1994, 0.2010), shape=(1, 1, 3))
#mean = tf.reshape((0.4914, 0.4822, 0.4465), shape=(1, 1, 3))

In [4]:
# Re-formatting the data so it fits the desired shapes by tf.keras.model

# train and validation split (60000 samples total in CIFAR-10)
x_train = np.zeros([50000, 32, 32, 3])
y_train = np.zeros([50000, 1])
x_val = np.zeros([10000, 32, 32, 3])
y_val = np.zeros([10000, 1])

i = 0
j = 0
for sample, label in ds["train"].take(60000):
  if i < 50000:
    x_train[i] = np.asarray(sample)
    y_train[i] = np.asarray(label)
  else: 
    x_val[j] = np.asarray(sample)
    y_val[j] = np.asarray(label)
    j += 1
  i += 1


In [5]:
# hyperparameters taken from cifar_training function in https://github.com/gahaalt/ResNets-in-tensorflow2/blob/master/cifar_training_tools.py
lr_boundaries=[400, 32000, 48000, 64000]
lr_values=[0.01, 0.1, 0.01, 0.001]
schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=lr_boundaries[:-1], values=lr_values)
opt = tf.keras.optimizers.SGD(schedule, momentum=0.9, nesterov=False)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.metrics.SparseCategoricalAccuracy()
model.compile(loss=loss_fn, optimizer=opt, metrics=metrics)

tf.keras.utils.plot_model(model)

#early_stopping = EarlyStopping(monitor='val_accuracy', verbose=1,
#                               patience=7,
#                               mode='max',
#                               restore_best_weights=True)

batch_size = 128
epochs = 5

#callbacks_list = [early_stopping]

history = model.fit(x_train, y_train, batch_size=batch_size,
                    validation_data=(x_val, y_val),
                    #steps_per_epoch=len(x_train) // batch_size,
                    #callbacks=callbacks_list,
                    epochs=epochs, verbose=1)

Epoch 1/5
391/391 [==============================] - 58s 148ms/step - loss: 2.5499 - sparse_categorical_accuracy: 0.4326 - val_loss: 4.3120 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/5
391/391 [==============================] - 57s 145ms/step - loss: 2.0498 - sparse_categorical_accuracy: 0.5760 - val_loss: 4.7467 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/5
391/391 [==============================] - 57s 145ms/step - loss: 1.4905 - sparse_categorical_accuracy: 0.7121 - val_loss: 3.3884 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/5
391/391 [==============================] - 57s 146ms/step - loss: 1.1893 - sparse_categorical_accuracy: 0.7736 - val_loss: 5.3283 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/5
391/391 [==============================] - 57s 145ms/step - loss: 1.0061 - sparse_categorical_accuracy: 0.8070 - val_loss: 4.5232 - val_sparse_categorical_accuracy: 0.0000e+00
